In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load the dataset
file_path = '/content/engine_data.csv'
data = pd.read_csv(file_path)

# Check for duplicates and drop them
data = data.drop_duplicates()

# Separate features and target
X = data.drop(columns=['Engine Condition'])
y = data['Engine Condition']

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=10)

# Q-Learning Setup
states = np.array(X_train)
n_states = states.shape[0]
n_actions = 2  # Assuming binary classification: 0 or 1

# Q-table initialization
q_table = np.zeros((n_states, n_actions))

# Hyperparameters
alpha = 0.1  # Learning rate
gamma = 0.9  # Discount factor
epsilon = 0.1  # Exploration rate
episodes = 100

# Reward function
def reward_function(state, action, true_label):
    if action == true_label:
        return 1  # Correct prediction
    else:
        return -1  # Incorrect prediction

# Training the agent
for episode in range(episodes):
    for i, state in enumerate(states):
        # Select action (epsilon-greedy)
        if np.random.rand() < epsilon:
            action = np.random.choice(n_actions)
        else:
            action = np.argmax(q_table[i])

        # Get the true label
        true_label = y_train.iloc[i]

        # Compute reward
        reward = reward_function(state, action, true_label)

        # Update Q-value
        q_table[i, action] = q_table[i, action] + alpha * (reward + gamma * np.max(q_table[i]) - q_table[i, action])

# Evaluate the agent
correct_predictions = 0
for i, state in enumerate(X_test):
    action = np.argmax(q_table[i % n_states])  # Predict based on Q-table
    if action == y_test.iloc[i]:
        correct_predictions += 1

accuracy = correct_predictions / len(X_test)
print(f"Reinforcement Learning Accuracy: {accuracy:.2f}")


Reinforcement Learning Accuracy: 0.55


In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

# Load the dataset
file_path = '/content/engine_data.csv'
data = pd.read_csv(file_path)

# Check for duplicates and drop them
data = data.drop_duplicates()

# Separate features and target
X = data.drop(columns=['Engine Condition'])
y = data['Engine Condition']

# Apply PCA to reduce dimensionality for better state representation
pca = PCA(n_components=5)  # Choose the number of components based on explained variance
X_reduced = pca.fit_transform(X)

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_reduced)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=10)

# Q-Learning Setup
n_states = X_train.shape[0]
n_actions = len(np.unique(y))  # Number of unique classes (e.g., binary classification = 2)

# Q-table initialization
q_table = np.zeros((n_states, n_actions))

# Hyperparameters
alpha = 0.1  # Learning rate
gamma = 0.9  # Discount factor
epsilon = 0.1  # Exploration rate
epsilon_decay = 0.99  # Gradually reduce exploration
episodes = 1000  # More episodes for training

# Reward function
def reward_function(prediction, true_label):
    return 1 if prediction == true_label else -1

# Training the agent
for episode in range(episodes):
    for i, state in enumerate(X_train):
        # Select action (epsilon-greedy)
        if np.random.rand() < epsilon:
            action = np.random.choice(n_actions)
        else:
            action = np.argmax(q_table[i])

        # Get the true label
        true_label = y_train.iloc[i]

        # Compute reward
        reward = reward_function(action, true_label)

        # Update Q-value
        q_table[i, action] = q_table[i, action] + alpha * (
            reward + gamma * np.max(q_table[i]) - q_table[i, action]
        )

    # Decay epsilon to reduce exploration over time
    epsilon = max(epsilon * epsilon_decay, 0.01)

# Evaluate the agent
correct_predictions = 0
for i, state in enumerate(X_test):
    action = np.argmax(q_table[i % n_states])  # Predict based on Q-table
    if action == y_test.iloc[i]:
        correct_predictions += 1

accuracy = correct_predictions / len(X_test)
print(f"Optimized Reinforcement Learning Accuracy: {accuracy:.2f}")

Optimized Reinforcement Learning Accuracy: 0.55
